# Tokenization Pipeline

```{figure} figs/tokenization_pipeline.png
---
width: 80%
name: fig-tokenization-pipeline
---
Tokenization pipeline
```

The tokenization process typically involves the following steps:

1. **Normalization**: This is the process of converting text into a standard format. This may involve converting all text to lowercase, removing punctuation or special characters, correcting spelling errors, and so on. The goal of normalization is to reduce the complexity of the text and to focus on the important features.

2. **Pre-tokenization**: This involves splitting the text into preliminary tokens. This could be as simple as splitting the text into words based on whitespace, or it could involve more complex processes like sentence segmentation.

3. **Model**: This is the core algorithm that determines how the pre-tokenized text is further broken down into tokens. There are various models available, including word-level, character-level, and subword-level tokenization models.

4. **Post-processing**: This involves any additional processing that needs to be done after tokenization. This could include removing stop words (common words like 'is', 'the', 'and', etc. that are often filtered out), stemming (reducing words to their root form), or lemmatization (reducing words to their base or dictionary form).

Let's now look at how we can implement this pipeline using Python.

## Implementing Tokenization in Python

We will be using the `transformers` library from Hugging Face, which provides a wide range of pre-trained models for NLP tasks. It also provides various tokenizers that we can use.

First, let's import the necessary libraries:


In [2]:
from transformers import AutoTokenizer


We can use the `AutoTokenizer` class to automatically infer the correct tokenizer to use based on the pre-trained model we specify.

### Normalization

Normalization is usually handled internally by the tokenizer. However, we can see the effect of normalization by directly calling the normalizer:


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))


hello how are u?


In this example, we're using the tokenizer for the `bert-base-uncased` model. The `normalize_str` method applies the normalizer to the input string, converting all characters to lowercase and removing any special characters.

### Pre-tokenization

Next, we perform pre-tokenization. This involves splitting the text into preliminary tokens:


In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('how', (7, 10)),
 ('are', (11, 14)),
 ('you', (16, 19)),
 ('?', (19, 20))]

Different models may use different pre-tokenization methods. For example, the GPT-2 and T5 models use different pre-tokenizers:


In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")

tokenizer = AutoTokenizer.from_pretrained("t5-small")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")


[('▁Hello,', (0, 6)),
 ('▁how', (7, 10)),
 ('▁are', (11, 14)),
 ('▁you?', (16, 20))]

### Model

The model is the core algorithm that determines how the pre-tokenized text is further broken down into tokens. This is usually handled internally by the tokenizer, so we won't go into detail here.

### Post-processing

Finally, we perform post-processing. This involves any additional processing that needs to be done after tokenization. For example, we might want to convert our tokens into input IDs, which can be inputted into our

model:


In [4]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('Ġhow', (6, 10)),
 ('Ġare', (10, 14)),
 ('Ġ', (14, 15)),
 ('Ġyou', (15, 19)),
 ('?', (19, 20))]

In this example, we're using the `pre_tokenize_str` method of the tokenizer's model to tokenize the input string.

### Postprocessing

Postprocessing involves any additional processing that needs to be done after the text has been tokenized. This can include padding the tokenized text to a certain length, adding special tokens (like start and end tokens), or converting the tokenized text into a format that can be used by a machine learning model.

Here's an example of how you can perform postprocessing using the `transformers` library:


In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize("Hello, how are you?")
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids

[7592, 1010, 2129, 2024, 2017, 1029]

In this example, the `convert_tokens_to_ids` method is used to convert each token into its corresponding input ID.

Another common post-processing step is adding special tokens, such as `[CLS]` and `[SEP]`, which are used by models like BERT:


In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize("Hello, how are you?")
tokens = ["[CLS]"] + tokens + ["[SEP]"]
tokens


['[CLS]', 'hello', ',', 'how', 'are', 'you', '?', '[SEP]']

In this example, we manually add the `[CLS]` token at the beginning of our token list and the `[SEP]` token at the end.

However, the `transformers` library provides a more convenient way to perform these post-processing steps:


In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
encoded_input = tokenizer("Hello, how are you?", add_special_tokens=True)
encoded_input


{'input_ids': [101, 7592, 1010, 2129, 2024, 2017, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In this example, the `tokenizer` method automatically tokenizes the input text, converts the tokens to input IDs, and adds the special tokens. The `add_special_tokens=True` argument specifies that we want to add the special tokens.

The `tokenizer` method returns an `Encoding` object, which contains all the information we need:


In [13]:
print(encoded_input.input_ids)  # The input IDs
print(encoded_input.token_type_ids)  # The token type IDs
print(encoded_input.attention_mask)  # The attention mask


[101, 7592, 1010, 2129, 2024, 2017, 1029, 102]
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1]


The `input_ids` attribute contains the input IDs, the `token_type_ids` attribute contains the token type IDs (used for models that have separate segments of input, like BERT), and the `attention_mask` attribute contains the attention mask (used for models that use attention, like BERT and GPT-2).
